## **PULSE SURVEY 11 MULTI SELECT VALIDATION** 

1. Check the Question Stem Total column for at least 3 single select questions
2. Check Count and Demographic Value Totals column for each demographic for at least 2 single select questions
    - Note that Reporting College and Multiple Ethnicities are double counting demographics, which means that each student with multiple majors / ethnicities is counted once in each unique category. So, a student in L&S and CNR are counted as 2 responses- one from L&S and one from CNR. This means that their Demographic Value Totals will add up to more than their Question Stem Totals
3. Check Count and Demographic Value Totals, by Undergrad Grad column for one non-double-counting demographic and one double-counting demographic for at least 2 single-select questions (preferably questions that haven’t been checked)
4. Check that each Question Stem Id matches their Question Stem/Item & Question Response
    - Use Pulse Survey Content documents for this (must download the .docx files to be able to view)
    - While you’re doing this, make sure the text looks correct
5. Repeat the same thing for multi select questions

### **Demographic Categories**
**Double counting**
- Reporting College
- Multiple Ethnicities

**Non-double-counting**
- Undergrad Grad
- Derived Residency Desc
- Entry Status Desc
- Ucb Level1 Ethnic Rollup Desc
- Ucb Level2 Ethnic Rollup Desc


In [291]:
from sklearn.pipeline import Pipeline, FeatureUnion
import pandas as pd
import numpy as np
from IPython.display import display

In [292]:
%run cleaning_transformers.ipynb

In [293]:
%run multiselect_counter_transformers.ipynb

In [294]:
DATA_SOURCE = pd.read_csv('11_ps_data_source.csv')
RAW_SURVEY = pd.read_csv('pulse_survey_11_raw_data.csv')

/var/folders/4p/hk2kpwld0sd7079v5yqmhzv00000gn/T/ipykernel_11603/1557138038.py:2: DtypeWarning: Columns (10,11,12,16,17,18,19,20,21,22,23,24,25,26,27,28,29,45,46,47,48,49,55,56,57,58,59,104,105) have mixed types. Specify dtype option on import or set low_memory=False.
  RAW_SURVEY = pd.read_csv('pulse_survey_11_raw_data.csv')


##
## 1. Clean raw data

In [295]:
# data cleaning variables
COLUMNS_TO_REMOVE = ['RecordedDate'] ## may need to add:'PHQ2SCORE', 'GAD2SCORE', 'PHQ2', 'GAD2'
UNGRAD_GRAD_COL = 'UNGRADGRADCD' ## may need to replace
RESIDENCY_COL = 'RESIDENCY' ## may need to replace
ENTRY_STATUS_COL = 'ENTRYSTATUSDESC' ## may need to replace
ETH_LEVEL1_COL = 'LEVEL1ETH' ## may need to replace
ETH_LEVEL2_COL = 'LEVEL2ETH' ## may need to replace
VALUES_TO_NULLIFY = [-99, '-99', -1, '-1', -999, '-999', 'Not selected'] ## may need to replace

############# OPTIONAL: use ONLY if Reporting College cols look like a stem id #############
# rename reporintg college columns to avoid them getting treated as a question
RAW_SURVEY = RAW_SURVEY.rename(columns={'REPORTCOLLEGE1':'Reporting College - First Plan',
                                        'REPORTCOLLEGE2':'Reporting College - Second Plan',
                                        'REPORTCOLLEGE3':'Reporting College - Third Plan'})
############################################################################################
COLLEGE_COLS = RAW_SURVEY.columns[RAW_SURVEY.columns.str.contains('Reporting College')]
MULTI_ETH_COLS = ['African American / Black',
                  'Asian / Asian American',
                  'Hispanic / Latinx',
                  'International',
                  'American Indian / Alaska Native',
                  'Pacific Islander',
                  'Southwest Asian / North African',
                  'White / Caucasian',
                  'No Response']

# counting variables
QUESTION_DESC = RAW_SURVEY.loc[[0]] 
DATA = RAW_SURVEY[1:] 
DEMOGRAPHIC_COLUMNS = ['Undergrad Grad',
                       'Derived Residency Desc',
                       'Entry Status Desc',
                       'Ucb Level1 Ethnic Rollup Desc',
                       'Ucb Level2 Ethnic Rollup Desc',
                       'Low-income Status',
                       'First Gen College',
                       'Person Gender Desc',
                       'Reporting College',
                       'Multiple Ethnicities']

cleaning_pipeline = Pipeline([
    # drop null responses, remove duplicates and columns, make all missing/irrelevant values nan
    ('null rows remover', RemoveNullRowsTransformer()),
    ('values nullifier', ReplaceValuesTransformer(values_to_replace=VALUES_TO_NULLIFY)),
    ('duplicates remover', RemoveFirstDuplicateTransformer()),
    ('irrelevant columns remover', RemoveColumnsTransformer(columns_to_remove=COLUMNS_TO_REMOVE)),
    # rename column names
    ('undergrad grad col renamer', RenameColumnTransformer(UNGRAD_GRAD_COL, 'Undergrad Grad')),
    ('residency col renamer', RenameColumnTransformer(RESIDENCY_COL, 'Derived Residency Desc')),
    ('entry status col renamer', RenameColumnTransformer(ENTRY_STATUS_COL, 'Entry Status Desc')),
    ('ethnic lvl1 col renamer', RenameColumnTransformer(ETH_LEVEL1_COL, 'Ucb Level1 Ethnic Rollup Desc')),
    ('ethnic lvl2 col renamer', RenameColumnTransformer(ETH_LEVEL2_COL, 'Ucb Level2 Ethnic Rollup Desc')),
    # rename dataframe values
    ('undergrad value renamer', RelabelColumnTransformer(column_to_relabel='Undergrad Grad', new_label='U')),
    ('grad value renamer', RelabelColumnTransformer(column_to_relabel='Undergrad Grad', new_label='G')),
    ('first-year entry value renamer', RelabelColumnTransformer(column_to_relabel='Entry Status Desc', new_label='First-year')),
    # replace ADVANCED STANDING with NaN for all grad students
    ('advanced standing grad nullifier', ReplaceStringWithNaNTransformer(standing_col='Entry Status Desc')),
    # create columns for double counting demographics & mental health scores
    ('reporting clg col generator', UniqueStringListTransformer(columns_to_list=COLLEGE_COLS, unique_col_list='Reporting College')),
    ('multiple eth col generator', UniqueStringListTransformer(columns_to_list=MULTI_ETH_COLS, unique_col_list='Multiple Ethnicities')),
    ('depression col generator', AddColumnsTransformer(column_1='MHLTH1', column_2='MHLTH2', new_column='PHQ2', binary_column='DEPRESSION')),
    ('anxiety col generator', AddColumnsTransformer(column_1='MHLTH3', column_2='MHLTH4', new_column='GAD2', binary_column='ANXIETY'))
])

In [296]:
RAW_SURVEY = cleaning_pipeline.fit_transform(DATA)

In [297]:
DATA_SOURCE['Count'] = pd.to_numeric(DATA_SOURCE['Count'], downcast="integer")
DATA_SOURCE.head(2)

,Question Stem Id,Question Item Id,Demographic Category,Demographic Value,Undergrad Grad,Question Response,Count,Question Item,Question Stem,Demographic Value Total,"Demographic Value Total, by Undergrad Grad",Question Stem Total,Question Item Total
0,ADV_UG,ADV_UG,Undergrad Grad,U,U,No,2269,NaN,"During this academic year (since the beginning of the Fall 21 semester), have you consulted with an academic advisor in your major or college?",5690,5690,5690,5690
1,ADV_UG,ADV_UG,Undergrad Grad,U,U,Yes,3421,NaN,"During this academic year (since the beginning of the Fall 21 semester), have you consulted with an academic advisor in your major or college?",5690,5690,5690,5690


In [298]:
RAW_SURVEY.head(2)

,ResponseId,EDUCNONEXAMLEVEL,EDUCNONEXAMLEVELCD,UGENTRYSTATUS,REGSTATUSDESC,GENDER,SHORTETHNICDESC,TYPE,Undergrad Grad,LowSocioEconomicStatusFlg,NeitherParent4yrClgDegFlg,Pulse10cmp,ACADPLANNM1,ACADPLANNM2,ACADPLANNM3,CNR,CHE,COE,CED,CLS,BUS,GSE,GSJ,SPP,SOI,LAW,OPT,SPH,SSW,ADV_UG,ADV_MODES_UG_1,ADV_MODES_UG_2,ADV_MODES_UG_3,ADV_MODES_UG_4,ADV_MODES_UG_5,ADV_MODES_UG_6,ADV_MODES_UG_7,ADV_LAST_UG,ADV_IMPACT_UG,ADV_BASIC_MODES_UG_1,ADV_BASIC_MODES_UG_2,ADV_BASIC_MODES_UG_3,ADV_BASIC_MODES_UG_4,ADV_BASIC_MODES_UG_5,ADV_BASIC_RANK_UG_1,ADV_BASIC_RANK_UG_2,ADV_BASIC_RANK_UG_3,ADV_BASIC_RANK_UG_4,ADV_BASIC_RANK_UG_5,ADV_COMPLEX_MODES_UG_1,ADV_COMPLEX_MODES_UG_2,ADV_COMPLEX_MODES_UG_3,ADV_COMPLEX_MODES_UG_4,ADV_COMPLEX_MODES_UG_5,ADV_COMPLEX_RANK_UG_1,ADV_COMPLEX_RANK_UG_2,ADV_COMPLEX_RANK_UG_3,ADV_COMPLEX_RANK_UG_4,ADV_COMPLEX_RANK_UG_5,ADV_MET_G,ADV_AMT_G,ADV_TYPICAL_G,ADV_RECENT_G,ADV_IMPACT_G,HOUS_INSEC,HOUS_PLACES_1,HOUS_PLACES_2,HOUS_PLACES_3,HOUS_PLACES_4,HOUS_PLACES_5,HOUS_PLACES_6,HOUS_PLACES_7,HOUS_PLACES_8,HOUS_PLACES_9,HOUS_PLACES_10,HOUS_PLACES_11,HOUS_PLACES_12,HOUS_AMT,HOUS_WORRY,HOUS_FAR,HOUS_COMMUTE,COMM_LEADERS,COVID_COMFORT_1,COVID_COMFORT_2,COVID_BEHAVIOR1_1,COVID_BEHAVIOR1_2,COVID_BEHAVIOR1_3,COVID_BEHAVIOR1_4,COVID_BEHAVIOR1_5,COVID_BEHAVIOR2_1,COVID_BEHAVIOR2_2,COVID_BEHAVIOR2_3,COVID_BEHAVIOR2_4,COVID_BEHAVIOR2_5,COVID_BEHAVIOR2_6,COVID_BEHAVIOR2_7,COVID_BEHAVIOR2_8,COVID_MASK_1,COVID_MASK_2,MHLTH1,MHLTH2,MHLTH3,MHLTH4,PHQ2SCORE,GAD2SCORE,PHQ2,GAD2,Semester Year Name Concat,African American / Black,Asian / Asian American,Hispanic / Latinx,International,American Indian / Alaska Native,Pacific Islander,Southwest Asian / North African,White / Caucasian,No Response,First Gen College,Person Gender Desc,Entry Status Desc,Derived Residency Desc,Ucb Level1 Ethnic Rollup Desc,Ucb Level2 Ethnic Rollup Desc,Reporting College - First Plan,Reporting College - Second Plan,Reporting College - Third Plan,Low-income Status,Reporting College,Multiple Ethnicities,DEPRESSION,ANXIETY
1,R_2c0AYPeQkvJtmJ2,Senior,Senior,Transfer,Continuing Student,Female,International,Undergraduate,U,NaN,NaN,0,Electrical Eng & Comp Sci BS,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022 Spring,NaN,Asian / Asian American,NaN,International,NaN,NaN,NaN,NaN,NaN,Not first-generation college,Woman,ADVANCED STANDING,International,International,International,College of Engineering,NaN,NaN,Not low-income,[College of Engineering],"[Asian / Asian American, International]",NaN,NaN
3,R_2rqJppW04QcQABh,Doctoral not advanced to candidacy,Doctoral (not advanced to candidacy),NaN,Continuing Student,Male,Chinese,Graduate Student,G,NaN,NaN,1,Materials Science & Eng PhD,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,One time,Email,Email,The mode of contact had a positive impact,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Never,1 mile to 2 miles,Rarely,Email,About the same comfort,A little more uncomfortable,Not important,Very important,Not important,Slightly important,Moderately important,Important,Slightly important,Moderately important,Slightly important,Slightly important,Slightly important,Important,Moderately important,Very comfortable,Somewhat comfortable,Several days,Not at all,Several days,Not at all,1.0,1.0,1.0,1.0,2022 Spring,NaN,Asian / Asian American,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not first-generation college,Man,FIRST TIME IN PROGRAM,CA Resident,Asian,Asian,College of Engineering,NaN,NaN,NaN,[College of Engineering],[Asian / Asian American],NO,NO


In [299]:
STEM_ID = DATA_SOURCE['Question Stem Id'].unique()
STEM_ID

array(['ADV_UG', 'ADV_MODES_UG_1', 'ADV_MODES_UG_2', 'ADV_MODES_UG_3',
       'ADV_MODES_UG_4', 'ADV_MODES_UG_5', 'ADV_MODES_UG_6',
       'ADV_MODES_UG_7', 'ADV_LAST_UG', 'ADV_IMPACT_UG',
       'ADV_BASIC_RANK_UG_1', 'ADV_BASIC_RANK_UG_2',
       'ADV_BASIC_RANK_UG_3', 'ADV_BASIC_RANK_UG_4',
       'ADV_BASIC_RANK_UG_5', 'ADV_COMPLEX_RANK_UG_1',
       'ADV_COMPLEX_RANK_UG_2', 'ADV_COMPLEX_RANK_UG_3',
       'ADV_COMPLEX_RANK_UG_4', 'ADV_COMPLEX_RANK_UG_5', 'ADV_MET_G',
       'ADV_AMT_G', 'ADV_TYPICAL_G', 'ADV_RECENT_G', 'ADV_IMPACT_G',
       'HOUS_INSEC', 'HOUS_AMT', 'HOUS_WORRY', 'HOUS_FAR', 'HOUS_COMMUTE',
       'COMM_LEADERS', 'COVID_COMFORT_1', 'COVID_COMFORT_2',
       'COVID_BEHAVIOR1_1', 'COVID_BEHAVIOR1_2', 'COVID_BEHAVIOR1_3',
       'COVID_BEHAVIOR1_4', 'COVID_BEHAVIOR1_5', 'COVID_BEHAVIOR2_1',
       'COVID_BEHAVIOR2_2', 'COVID_BEHAVIOR2_3', 'COVID_BEHAVIOR2_4',
       'COVID_BEHAVIOR2_5', 'COVID_BEHAVIOR2_6', 'COVID_BEHAVIOR2_7',
       'COVID_BEHAVIOR2_8', 'COVID_MAS

In [300]:
MULTI_SELECT = ['ADV_BASIC_MODES_UG', 'ADV_COMPLEX_MODES_UG', 'HOUS_PLACES']

SINGLE_SELECT = [id for id in STEM_ID if id not in MULTI_SELECT]

SINGLE_DEMOS = ['Undergrad Grad', 'Derived Residency Desc', 
             'Entry Status Desc', 'Ucb Level1 Ethnic Rollup Desc',
             'Ucb Level2 Ethnic Rollup Desc', 'Low-income Status', 
             'First Gen College', 'Person Gender Desc']

DOUBLE_DEMOS = ['Multiple Ethnicities ', 'Reporting College']

##
## 2. Check the Question Stem Total column for at least 3 single select questions

In [301]:
# completed function 
def check_qstem_total(qstems): 
    for qstem in qstems: 
        print('_____', qstem, '_____')
        # finding data source value for question stem total 
        allstemtotal = DATA_SOURCE[DATA_SOURCE['Question Stem Id'].str.contains(qstem, case=False)]
        stemtotal = allstemtotal[['Question Item Id', 'Question Stem Total']]
        stemtotal = stemtotal.drop_duplicates(ignore_index=True)
        if len(stemtotal['Question Stem Total'].value_counts()) > 1: 
            print('ERROR: DATA SOURCE HAS MULTIPLE STEM TOTAL VALUES -- CHECK .value_counts()')
            display(allstemtotal[['Question Item Id', 'Demographic Category', 'Question Stem Total']].drop_duplicates(ignore_index=True))
            
        data_source_val = stemtotal['Question Stem Total'][0]

        #finding raw survey value for question stem total 
        stems = [stem for stem in RAW_SURVEY.columns if qstem in stem]
        raw_survey_val = len(RAW_SURVEY[stems].dropna(how='all'))

        print('DATA SOURCE:', data_source_val)
        print('RAW SURVEY:', raw_survey_val)
        print('Equal?:', data_source_val == raw_survey_val) 
        print("\n")
        

# check multiple stem totals 
qstems = MULTI_SELECT
check_qstem_total(qstems)

_____ ADV_BASIC_MODES_UG _____
ERROR: DATA SOURCE HAS MULTIPLE STEM TOTAL VALUES -- CHECK .value_counts()


,Question Item Id,Demographic Category,Question Stem Total
0,ADV_BASIC_MODES_UG_1,Reporting College,5595
1,ADV_BASIC_MODES_UG_2,Reporting College,5595
2,ADV_BASIC_MODES_UG_3,Reporting College,5595
3,ADV_BASIC_MODES_UG_4,Reporting College,5595
4,ADV_BASIC_MODES_UG_5,Reporting College,5595
5,ADV_BASIC_MODES_UG_1,Multiple Ethnicities,7148
6,ADV_BASIC_MODES_UG_2,Multiple Ethnicities,7148
7,ADV_BASIC_MODES_UG_3,Multiple Ethnicities,7148
8,ADV_BASIC_MODES_UG_4,Multiple Ethnicities,7148
9,ADV_BASIC_MODES_UG_5,Multiple Ethnicities,7148


DATA SOURCE: 5595
RAW SURVEY: 5518
Equal?: False


_____ ADV_COMPLEX_MODES_UG _____
ERROR: DATA SOURCE HAS MULTIPLE STEM TOTAL VALUES -- CHECK .value_counts()


,Question Item Id,Demographic Category,Question Stem Total
0,ADV_COMPLEX_MODES_UG_1,Reporting College,5510
1,ADV_COMPLEX_MODES_UG_2,Reporting College,5510
2,ADV_COMPLEX_MODES_UG_3,Reporting College,5510
3,ADV_COMPLEX_MODES_UG_4,Reporting College,5510
4,ADV_COMPLEX_MODES_UG_5,Reporting College,5510
5,ADV_COMPLEX_MODES_UG_1,Multiple Ethnicities,7046
6,ADV_COMPLEX_MODES_UG_2,Multiple Ethnicities,7046
7,ADV_COMPLEX_MODES_UG_3,Multiple Ethnicities,7046
8,ADV_COMPLEX_MODES_UG_4,Multiple Ethnicities,7046
9,ADV_COMPLEX_MODES_UG_5,Multiple Ethnicities,7046


DATA SOURCE: 5510
RAW SURVEY: 5435
Equal?: False


_____ HOUS_PLACES _____
ERROR: DATA SOURCE HAS MULTIPLE STEM TOTAL VALUES -- CHECK .value_counts()


,Question Item Id,Demographic Category,Question Stem Total
0,HOUS_PLACES_1,Reporting College,687
1,HOUS_PLACES_2,Reporting College,687
2,HOUS_PLACES_3,Reporting College,687
3,HOUS_PLACES_4,Reporting College,687
4,HOUS_PLACES_5,Reporting College,687
...,...,...,...
115,HOUS_PLACES_12,Ucb Level1 Ethnic Rollup Desc,676
116,HOUS_PLACES_12,Ucb Level2 Ethnic Rollup Desc,676
117,HOUS_PLACES_12,Low-income Status,676
118,HOUS_PLACES_12,First Gen College,676


DATA SOURCE: 687
RAW SURVEY: 676
Equal?: False




In [302]:
qstem = MULTI_SELECT[0]
qstem

'ADV_BASIC_MODES_UG'

####
#### DATA SOURCE STEM TOTAL 

In [303]:
allstemtotal = DATA_SOURCE[DATA_SOURCE['Question Stem Id'].str.contains(qstem, case=False)]
allstemtotal.head(2)

,Question Stem Id,Question Item Id,Demographic Category,Demographic Value,Undergrad Grad,Question Response,Count,Question Item,Question Stem,Demographic Value Total,"Demographic Value Total, by Undergrad Grad",Question Stem Total,Question Item Total
20514,ADV_BASIC_MODES_UG,ADV_BASIC_MODES_UG_1,Reporting College,College of Chemistry,U,Selected,97,NaN,"Which of the following are acceptable modes of accessing academic advising for STRAIGHTFORWARD TASKS? Select all that apply. - Help/reception desk, in-person",197,197,5595,2936
20515,ADV_BASIC_MODES_UG,ADV_BASIC_MODES_UG_1,Reporting College,College of Engineering,U,Selected,355,NaN,"Which of the following are acceptable modes of accessing academic advising for STRAIGHTFORWARD TASKS? Select all that apply. - Help/reception desk, in-person",635,635,5595,2936


In [304]:
qstem = 'ADV_BASIC_MODES_UG'
allstemtotal = DATA_SOURCE[DATA_SOURCE['Question Stem Id'].str.contains(qstem, case=False)]
stemtotal = allstemtotal[['Question Item Id', 'Question Stem Total']]
stemtotal = stemtotal.drop_duplicates(ignore_index=True)
if len(stemtotal['Question Stem Total'].value_counts()) > 1: 
    print('ERROR: DATA SOURCE HAS MULTIPLE STEM TOTAL VALUES -- CHECK .value_counts()')
            
stemtotal

ERROR: DATA SOURCE HAS MULTIPLE STEM TOTAL VALUES -- CHECK .value_counts()


,Question Item Id,Question Stem Total
0,ADV_BASIC_MODES_UG_1,5595
1,ADV_BASIC_MODES_UG_2,5595
2,ADV_BASIC_MODES_UG_3,5595
3,ADV_BASIC_MODES_UG_4,5595
4,ADV_BASIC_MODES_UG_5,5595
5,ADV_BASIC_MODES_UG_1,7148
6,ADV_BASIC_MODES_UG_2,7148
7,ADV_BASIC_MODES_UG_3,7148
8,ADV_BASIC_MODES_UG_4,7148
9,ADV_BASIC_MODES_UG_5,7148


####
#### RAW SURVEY STEM TOTAL

In [305]:
# get all column names that have qstem 
stems = [stem for stem in RAW_SURVEY.columns if qstem in stem]

In [306]:
len(RAW_SURVEY[stems].dropna(how='all'))

5518

##
## 3. Check Count and Demographic Value Totals column for each demographic

In [307]:
# completed function (one demographic value) 
def check_count_onedemo(qstem, demo, double_count_demo = False): 
    # finding data source values #
    ds_counts = DATA_SOURCE[DATA_SOURCE['Question Stem Id']== qstem]
    ds_counts = ds_counts[ds_counts['Demographic Category'] == demo][['Demographic Value', 'Demographic Value Total', 'Undergrad Grad', 'Count', 'Question Response', 'Question Item Id']]
    ds_counts = ds_counts.sort_index(axis=1).sort_values(by = ['Demographic Value', 'Undergrad Grad', 'Count', 'Question Response']).reset_index(drop=True)
    
    # finding raw survey values #
    stems = [stem for stem in RAW_SURVEY.columns if qstem in stem]
    raw_final = pd.DataFrame() 
    for stem in stems: 
        raw = RAW_SURVEY
        if double_count_demo: 
            raw = RAW_SURVEY.explode(demo)
        raw['ID DUPLICATE'] = raw[stem]
        raw_piv = pd.pivot_table(raw, values=stem, index=['Undergrad Grad', demo, 'ID DUPLICATE'], aggfunc='count')
        raw_piv = raw_piv.reset_index().rename(columns={'Ungrad Grad Cd': 'Undergrad Grad', demo: 'Demographic Value', stem: 'Count', 'ID DUPLICATE': 'Question Response'})

        # make demographic value total col 
        select = [stem for stem in raw.columns if qstem in stem] + [demo]
        selected = raw[select]
        selected = selected.dropna(subset=stems, thresh = 1)
        demoval_total = selected[demo].value_counts().to_dict()
        raw_piv['Demographic Value Total'] = raw_piv['Demographic Value'].map(demoval_total)
        
        raw_piv['Question Item Id'] = [stem] * len(raw_piv) 
        raw_final = pd.concat([raw_final, raw_piv], ignore_index=True).sort_index(axis=1).sort_values(by = ['Demographic Value', 'Undergrad Grad', 'Count', 'Question Response']).reset_index(drop=True)
    

    print('DATA SOURCE: ')
    display(ds_counts)
    print("\n")
    print('RAW SURVEY: ')
    display(raw_final)


# completed function (all demographic values for ONE QUESTION ITEM) 
def check_count_alldemo(qitem, demo_vals): 
    for demo in demo_vals:
        print('DEMOGRAPHIC VALUE:', demo) 
        if demo in ['Reporting College', 'Multiple Ethnicities']:
            check_count_onedemo(qitem, demo, double_count_demo = True)
        else:
            check_count_onedemo(qitem, demo) 
        print("\n")
        
demo_cat = [#'Undergrad Grad',
            'Derived Residency Desc',
            'Entry Status Desc',
            'Ucb Level1 Ethnic Rollup Desc',
            'Ucb Level2 Ethnic Rollup Desc',
            'Low-income Status',
            'First Gen College',
            'Person Gender Desc',
            'Reporting College',
            'Multiple Ethnicities']

# if FALSE, check dataframes below by replacing the variables qitem and demo (typically because of cleaning/low counts) 

In [308]:
check_count_alldemo(MULTI_SELECT[0], demo_cat) 

DEMOGRAPHIC VALUE: Derived Residency Desc
DATA SOURCE: 


,Count,Demographic Value,Demographic Value Total,Question Item Id,Question Response,Undergrad Grad
0,2242,CA Resident,4269,ADV_BASIC_MODES_UG_1,Selected,U
1,2631,CA Resident,4269,ADV_BASIC_MODES_UG_3,Selected,U
2,2767,CA Resident,4269,ADV_BASIC_MODES_UG_2,Selected,U
3,2957,CA Resident,4269,ADV_BASIC_MODES_UG_5,Selected,U
4,3174,CA Resident,4269,ADV_BASIC_MODES_UG_4,Selected,U
5,278,International,595,ADV_BASIC_MODES_UG_1,Selected,U
6,348,International,595,ADV_BASIC_MODES_UG_3,Selected,U
7,349,International,595,ADV_BASIC_MODES_UG_2,Selected,U
8,349,International,595,ADV_BASIC_MODES_UG_5,Selected,U
9,402,International,595,ADV_BASIC_MODES_UG_4,Selected,U




RAW SURVEY: 


,Count,Demographic Value,Demographic Value Total,Question Item Id,Question Response,Undergrad Grad
0,2242,CA Resident,4269,ADV_BASIC_MODES_UG_1,Selected,U
1,2631,CA Resident,4269,ADV_BASIC_MODES_UG_3,Selected,U
2,2767,CA Resident,4269,ADV_BASIC_MODES_UG_2,Selected,U
3,2957,CA Resident,4269,ADV_BASIC_MODES_UG_5,Selected,U
4,3174,CA Resident,4269,ADV_BASIC_MODES_UG_4,Selected,U
5,278,International,595,ADV_BASIC_MODES_UG_1,Selected,U
6,348,International,595,ADV_BASIC_MODES_UG_3,Selected,U
7,349,International,595,ADV_BASIC_MODES_UG_2,Selected,U
8,349,International,595,ADV_BASIC_MODES_UG_5,Selected,U
9,402,International,595,ADV_BASIC_MODES_UG_4,Selected,U




DEMOGRAPHIC VALUE: Entry Status Desc
DATA SOURCE: 


...

,Count,Demographic Value,Demographic Value Total,Question Item Id,Question Response,Undergrad Grad
0,97,College of Chemistry,197,ADV_BASIC_MODES_UG_1,Selected,U
1,113,College of Chemistry,197,ADV_BASIC_MODES_UG_2,Selected,U
2,124,College of Chemistry,197,ADV_BASIC_MODES_UG_3,Selected,U
3,144,College of Chemistry,197,ADV_BASIC_MODES_UG_4,Selected,U
4,153,College of Chemistry,197,ADV_BASIC_MODES_UG_5,Selected,U
5,355,College of Engineering,635,ADV_BASIC_MODES_UG_1,Selected,U
6,367,College of Engineering,635,ADV_BASIC_MODES_UG_2,Selected,U
7,426,College of Engineering,635,ADV_BASIC_MODES_UG_3,Selected,U
8,455,College of Engineering,635,ADV_BASIC_MODES_UG_5,Selected,U
9,468,College of Engineering,635,ADV_BASIC_MODES_UG_4,Selected,U




DEMOGRAPHIC VALUE: Multiple Ethnicities
DATA SOURCE: 


,Count,Demographic Value,Demographic Value Total,Question Item Id,Question Response,Undergrad Grad
0,96,African American / Black,205,ADV_BASIC_MODES_UG_1,Selected,U
1,121,African American / Black,205,ADV_BASIC_MODES_UG_3,Selected,U
2,127,African American / Black,205,ADV_BASIC_MODES_UG_2,Selected,U
3,127,African American / Black,205,ADV_BASIC_MODES_UG_5,Selected,U
4,152,African American / Black,205,ADV_BASIC_MODES_UG_4,Selected,U
5,34,American Indian / Alaska Native,74,ADV_BASIC_MODES_UG_1,Selected,U
6,45,American Indian / Alaska Native,74,ADV_BASIC_MODES_UG_3,Selected,U
7,48,American Indian / Alaska Native,74,ADV_BASIC_MODES_UG_2,Selected,U
8,50,American Indian / Alaska Native,74,ADV_BASIC_MODES_UG_5,Selected,U
9,55,American Indian / Alaska Native,74,ADV_BASIC_MODES_UG_4,Selected,U




RAW SURVEY: 


,Count,Demographic Value,Demographic Value Total,Question Item Id,Question Response,Undergrad Grad
0,96,African American / Black,205,ADV_BASIC_MODES_UG_1,Selected,U
1,121,African American / Black,205,ADV_BASIC_MODES_UG_3,Selected,U
2,127,African American / Black,205,ADV_BASIC_MODES_UG_2,Selected,U
3,127,African American / Black,205,ADV_BASIC_MODES_UG_5,Selected,U
4,152,African American / Black,205,ADV_BASIC_MODES_UG_4,Selected,U
5,34,American Indian / Alaska Native,74,ADV_BASIC_MODES_UG_1,Selected,U
6,45,American Indian / Alaska Native,74,ADV_BASIC_MODES_UG_3,Selected,U
7,48,American Indian / Alaska Native,74,ADV_BASIC_MODES_UG_2,Selected,U
8,50,American Indian / Alaska Native,74,ADV_BASIC_MODES_UG_5,Selected,U
9,55,American Indian / Alaska Native,74,ADV_BASIC_MODES_UG_4,Selected,U


In [309]:
qstem = MULTI_SELECT[0]
demo = 'Derived Residency Desc'

####
#### DATA SOURCE COUNTS DF

In [310]:
DATA_SOURCE.head(3)

,Question Stem Id,Question Item Id,Demographic Category,Demographic Value,Undergrad Grad,Question Response,Count,Question Item,Question Stem,Demographic Value Total,"Demographic Value Total, by Undergrad Grad",Question Stem Total,Question Item Total
0,ADV_UG,ADV_UG,Undergrad Grad,U,U,No,2269,NaN,"During this academic year (since the beginning of the Fall 21 semester), have you consulted with an academic advisor in your major or college?",5690,5690,5690,5690
1,ADV_UG,ADV_UG,Undergrad Grad,U,U,Yes,3421,NaN,"During this academic year (since the beginning of the Fall 21 semester), have you consulted with an academic advisor in your major or college?",5690,5690,5690,5690
2,ADV_MODES_UG_1,ADV_MODES_UG_1,Undergrad Grad,U,U,0/None,2607,NaN,"During this academic year (since the beginning of the Fall 21 semester), have you consulted with an academic advisor in your major or college? - Help/reception desk, in-person",3421,3421,3421,3421


In [311]:
ds_counts = DATA_SOURCE[DATA_SOURCE['Question Stem Id']== qstem]
ds_counts = ds_counts[ds_counts['Demographic Category'] == demo][['Demographic Value', 'Demographic Value Total', 'Undergrad Grad', 'Count', 'Question Response', 'Question Item Id']]
ds_counts = ds_counts.sort_index(axis=1).sort_values(by = ['Demographic Value', 'Undergrad Grad', 'Count', 'Question Response']).reset_index(drop=True)
ds_counts

,Count,Demographic Value,Demographic Value Total,Question Item Id,Question Response,Undergrad Grad
0,2242,CA Resident,4269,ADV_BASIC_MODES_UG_1,Selected,U
1,2631,CA Resident,4269,ADV_BASIC_MODES_UG_3,Selected,U
2,2767,CA Resident,4269,ADV_BASIC_MODES_UG_2,Selected,U
3,2957,CA Resident,4269,ADV_BASIC_MODES_UG_5,Selected,U
4,3174,CA Resident,4269,ADV_BASIC_MODES_UG_4,Selected,U
5,278,International,595,ADV_BASIC_MODES_UG_1,Selected,U
6,348,International,595,ADV_BASIC_MODES_UG_3,Selected,U
7,349,International,595,ADV_BASIC_MODES_UG_2,Selected,U
8,349,International,595,ADV_BASIC_MODES_UG_5,Selected,U
9,402,International,595,ADV_BASIC_MODES_UG_4,Selected,U


####
#### RAW SURVEY COUNTS DF

In [312]:
stems = [stem for stem in RAW_SURVEY.columns if qstem in stem]
raw_final = pd.DataFrame() 
for stem in stems: 
    # uncomment line below if double counting
    # RAW_SURVEY = RAW_SURVEY.explode(# insert double counting demographic value)
    RAW_SURVEY['ID DUPLICATE'] = RAW_SURVEY[stem]
    raw_piv = pd.pivot_table(RAW_SURVEY, values=stem, index=['Undergrad Grad', demo, 'ID DUPLICATE'], aggfunc='count')

    raw_piv = raw_piv.reset_index().rename(columns={'Ungrad Grad Cd': 'Undergrad Grad', demo: 'Demographic Value', stem: 'Count', 'ID DUPLICATE': 'Question Response'})

    #make demographic value total col 
    select = [stem for stem in RAW_SURVEY.columns if qstem in stem] + ['Derived Residency Desc']
    selected = RAW_SURVEY[select]
    selected = selected.dropna(subset=stems, thresh = 1)
    demoval_total = selected['Derived Residency Desc'].value_counts().to_dict()
    raw_piv['Demographic Value Total'] = raw_piv['Demographic Value'].map(demoval_total)

    #replace low counts with -1
    #raw_piv['Count'] = raw_piv['Count'].apply(lambda x: -1 if x < 11 else x)
    
    raw_piv['Question Item Id'] = [stem] * len(raw_piv) 
    raw_piv = raw_piv.sort_index(axis=1).sort_values(by = ['Demographic Value', 'Undergrad Grad', 'Count', 'Question Response']).reset_index(drop=True)
    raw_final = pd.concat([raw_final, raw_piv], ignore_index=True) 
    
raw_final

,Count,Demographic Value,Demographic Value Total,Question Item Id,Question Response,Undergrad Grad
0,2242,CA Resident,4269,ADV_BASIC_MODES_UG_1,Selected,U
1,278,International,595,ADV_BASIC_MODES_UG_1,Selected,U
2,377,Out of State Domestic,654,ADV_BASIC_MODES_UG_1,Selected,U
3,2767,CA Resident,4269,ADV_BASIC_MODES_UG_2,Selected,U
4,349,International,595,ADV_BASIC_MODES_UG_2,Selected,U
5,430,Out of State Domestic,654,ADV_BASIC_MODES_UG_2,Selected,U
6,2631,CA Resident,4269,ADV_BASIC_MODES_UG_3,Selected,U
7,348,International,595,ADV_BASIC_MODES_UG_3,Selected,U
8,412,Out of State Domestic,654,ADV_BASIC_MODES_UG_3,Selected,U
9,3174,CA Resident,4269,ADV_BASIC_MODES_UG_4,Selected,U


In [313]:
ds_counts.astype(str).equals(raw_piv.astype(str))

False

##
## 4. Check Count and Demographic Value Totals, by Undergrad Grad column for one non-double-counting demographic and one double-counting demographic for at least 2 single-select questions 
Preferably questions that haven’t been checked

In [317]:
# completed function (one demographic value) 
def check_count_ug_onedemo(qstem, demo, double_count_demo = False): 
    # finding data source values #
    ds_counts = DATA_SOURCE[DATA_SOURCE['Question Stem Id']== qstem]
    ds_counts = ds_counts[ds_counts['Demographic Category'] == demo][['Demographic Value', 'Demographic Value Total, by Undergrad Grad', 'Undergrad Grad', 'Count', 'Question Response', 'Question Item Id']]
    ds_counts = ds_counts.sort_index(axis=1).sort_values(by = ['Demographic Value', 'Undergrad Grad', 'Count', 'Question Response']).reset_index(drop=True)
    
    # finding raw survey values #
    stems = [stem for stem in RAW_SURVEY.columns if qstem in stem]
    raw_final = pd.DataFrame() 
    for stem in stems: 
        raw = RAW_SURVEY
        if double_count_demo: 
            raw = RAW_SURVEY.explode(demo)
        raw['ID DUPLICATE'] = raw[stem]
        raw_piv = pd.pivot_table(raw, values=stem, index=['Undergrad Grad', demo, 'ID DUPLICATE'], aggfunc='count')
        raw_piv = raw_piv.reset_index().rename(columns={'Ungrad Grad Cd': 'Undergrad Grad', demo: 'Demographic Value', stem: 'Count', 'ID DUPLICATE': 'Question Response'})

        # make demographic value total by ug col 
         #make demographic value total col 
        select = [stem for stem in RAW_SURVEY.columns if qstem in stem] + [demo, 'Undergrad Grad']
        selected = RAW_SURVEY[select]
        selected = selected.dropna(subset=stems, thresh = 1)
        selected = selected[[demo, 'Undergrad Grad']].value_counts().to_frame().reset_index().rename(columns={0: 'Demographic Value Total, by Undergrad Grad', demo: 'Demographic Value'})
        raw_piv = selected.merge(raw_piv, 'right', on=['Demographic Value', 'Undergrad Grad'])

   
        raw_piv['Question Item Id'] = [stem] * len(raw_piv) 
        raw_piv = raw_piv.sort_index(axis=1)
        raw_final = pd.concat([raw_final, raw_piv], ignore_index=True).sort_values(by = ['Demographic Value', 'Undergrad Grad', 'Count', 'Question Response']).reset_index(drop=True)

    print('DATA SOURCE: ')
    display(ds_counts)
    print("\n")
    print('RAW SURVEY: ')
    display(raw_final)
    

# completed function (demographic values for ONE QUESTION ITEM) 
def check_count_ug_alldemo(qstem, demo_vals): 
    for demo in demo_vals:
        print('DEMOGRAPHIC VALUE:', demo) 
        if demo in ['Reporting College', 'Multiple Ethnicities']:
            check_count_ug_onedemo(qstem, demo, double_count_demo = True)
        else:
            check_count_ug_onedemo(qstem, demo) 
        print("\n")

In [319]:
check_count_ug_onedemo(qstem, demo)

DATA SOURCE: 


,Count,Demographic Value,"Demographic Value Total, by Undergrad Grad",Question Item Id,Question Response,Undergrad Grad
0,2242,CA Resident,4269,ADV_BASIC_MODES_UG_1,Selected,U
1,2631,CA Resident,4269,ADV_BASIC_MODES_UG_3,Selected,U
2,2767,CA Resident,4269,ADV_BASIC_MODES_UG_2,Selected,U
3,2957,CA Resident,4269,ADV_BASIC_MODES_UG_5,Selected,U
4,3174,CA Resident,4269,ADV_BASIC_MODES_UG_4,Selected,U
5,278,International,595,ADV_BASIC_MODES_UG_1,Selected,U
6,348,International,595,ADV_BASIC_MODES_UG_3,Selected,U
7,349,International,595,ADV_BASIC_MODES_UG_2,Selected,U
8,349,International,595,ADV_BASIC_MODES_UG_5,Selected,U
9,402,International,595,ADV_BASIC_MODES_UG_4,Selected,U




RAW SURVEY: 


,Count,Demographic Value,"Demographic Value Total, by Undergrad Grad",Question Item Id,Question Response,Undergrad Grad
0,2242,CA Resident,4269,ADV_BASIC_MODES_UG_1,Selected,U
1,2631,CA Resident,4269,ADV_BASIC_MODES_UG_3,Selected,U
2,2767,CA Resident,4269,ADV_BASIC_MODES_UG_2,Selected,U
3,2957,CA Resident,4269,ADV_BASIC_MODES_UG_5,Selected,U
4,3174,CA Resident,4269,ADV_BASIC_MODES_UG_4,Selected,U
5,278,International,595,ADV_BASIC_MODES_UG_1,Selected,U
6,348,International,595,ADV_BASIC_MODES_UG_3,Selected,U
7,349,International,595,ADV_BASIC_MODES_UG_2,Selected,U
8,349,International,595,ADV_BASIC_MODES_UG_5,Selected,U
9,402,International,595,ADV_BASIC_MODES_UG_4,Selected,U


In [232]:
qstem = MULTI_SELECT[1]
demo = 'Person Gender Desc'#np.random.choice(demo)


####
#### DATA SOURCE COUNTS DF BY UG

In [229]:
ds_counts = DATA_SOURCE[DATA_SOURCE['Question Stem Id']== qstem]
ds_counts = ds_counts[ds_counts['Demographic Category'] == demo][['Demographic Value', 'Demographic Value Total, by Undergrad Grad', 'Undergrad Grad', 'Count', 'Question Response', 'Question Item Id']]
ds_counts = ds_counts.sort_index(axis=1).sort_values(by = ['Demographic Value', 'Undergrad Grad', 'Count', 'Question Response']).reset_index(drop=True)
ds_counts

,Count,Demographic Value,"Demographic Value Total, by Undergrad Grad",Question Item Id,Question Response,Undergrad Grad
0,34,Decline to State,138,ADV_COMPLEX_MODES_UG_2,Selected,U
1,35,Decline to State,138,ADV_COMPLEX_MODES_UG_1,Selected,U
2,51,Decline to State,138,ADV_COMPLEX_MODES_UG_5,Selected,U
3,100,Decline to State,138,ADV_COMPLEX_MODES_UG_4,Selected,U
4,105,Decline to State,138,ADV_COMPLEX_MODES_UG_3,Selected,U
5,-1,Different Identity,-1,ADV_COMPLEX_MODES_UG_1,Selected,U
6,-1,Different Identity,-1,ADV_COMPLEX_MODES_UG_2,Selected,U
7,-1,Different Identity,-1,ADV_COMPLEX_MODES_UG_3,Selected,U
8,-1,Different Identity,-1,ADV_COMPLEX_MODES_UG_4,Selected,U
9,-1,Different Identity,-1,ADV_COMPLEX_MODES_UG_5,Selected,U


####
#### RAW SURVEY COUNTS DF BY UG

In [280]:
stems = [stem for stem in RAW_SURVEY.columns if qstem in stem]
raw_final = pd.DataFrame() 
for stem in stems: 
    # uncomment line below if double counting
    # RAW_SURVEY = RAW_SURVEY.explode(# insert double counting demographic value)
    RAW_SURVEY['ID DUPLICATE'] = RAW_SURVEY[stem]
    raw_piv = pd.pivot_table(RAW_SURVEY, values=stem, index=['Undergrad Grad', demo, 'ID DUPLICATE'], aggfunc='count')

    raw_piv = raw_piv.reset_index().rename(columns={'Ungrad Grad Cd': 'Undergrad Grad', demo: 'Demographic Value', stem: 'Count', 'ID DUPLICATE': 'Question Response'})

    #make demographic value total col 
    select = [stem for stem in RAW_SURVEY.columns if qstem in stem] + [demo, 'Undergrad Grad']
    selected = RAW_SURVEY[select]
    selected = selected.dropna(subset=stems, thresh = 1)
    selected = selected[[demo, 'Undergrad Grad']].value_counts().to_frame().reset_index().rename(columns={0: 'Demographic Value Total, by Undergrad Grad', demo: 'Demographic Value'})
    raw_piv = selected.merge(raw_piv, 'right', on=['Demographic Value', 'Undergrad Grad'])
    
    
    raw_piv['Question Item Id'] = [stem] * len(raw_piv) 
    raw_piv = raw_piv.sort_index(axis=1).sort_values(by = ['Demographic Value', 'Undergrad Grad', 'Count', 'Question Response']).reset_index(drop=True)
    raw_final = pd.concat([raw_final, raw_piv], ignore_index=True).sort_values(by = ['Demographic Value', 'Undergrad Grad', 'Count', 'Question Response']).reset_index(drop=True)
    
raw_final

,Count,Demographic Value,"Demographic Value Total, by Undergrad Grad",Question Item Id,Question Response,Undergrad Grad
0,34,Decline to State,138,ADV_COMPLEX_MODES_UG_2,Selected,U
1,35,Decline to State,138,ADV_COMPLEX_MODES_UG_1,Selected,U
2,51,Decline to State,138,ADV_COMPLEX_MODES_UG_5,Selected,U
3,100,Decline to State,138,ADV_COMPLEX_MODES_UG_4,Selected,U
4,105,Decline to State,138,ADV_COMPLEX_MODES_UG_3,Selected,U
5,2,Different Identity,4,ADV_COMPLEX_MODES_UG_1,Selected,U
6,2,Different Identity,4,ADV_COMPLEX_MODES_UG_2,Selected,U
7,2,Different Identity,4,ADV_COMPLEX_MODES_UG_5,Selected,U
8,4,Different Identity,4,ADV_COMPLEX_MODES_UG_3,Selected,U
9,4,Different Identity,4,ADV_COMPLEX_MODES_UG_4,Selected,U


##
## 5. Check that each Question Stem Id matches their Question Stem/Item & Question Response

In [321]:
def check_qstem_qitem(): 
    STEM_IDS = DATA_SOURCE['Question Stem Id'].unique()
    for qstem in MULTI_SELECT: 
        qstem_str = DATA_SOURCE[DATA_SOURCE['Question Item Id'].str.contains(qstem, case=False)]['Question Stem'].unique()
        qitem_str = DATA_SOURCE[DATA_SOURCE['Question Item Id'].str.contains(qstem, case=False)]['Question Item'].unique()
        

        print('########', qstem, '########')
        print('QUESTION STEM:', qstem_str)
        print("\n")
        print('QUESTION ITEM:', qitem_str)
        print("\n")
    
check_qstem_qitem()

######## ADV_BASIC_MODES_UG ########
QUESTION STEM: ['Which of the following are acceptable modes of accessing academic advising for STRAIGHTFORWARD TASKS?  Select all that apply. -  Help/reception desk, in-person'
 'Which of the following are acceptable modes of accessing academic advising for STRAIGHTFORWARD TASKS?  Select all that apply. -  Help desk by Zoom or phone (remote)'
 'Which of the following are acceptable modes of accessing academic advising for STRAIGHTFORWARD TASKS?  Select all that apply. -  One-on-one meeting, in-person'
 'Which of the following are acceptable modes of accessing academic advising for STRAIGHTFORWARD TASKS?  Select all that apply. -  One-on-one meeting, by Zoom or phone (remote)'
 'Which of the following are acceptable modes of accessing academic advising for STRAIGHTFORWARD TASKS?  Select all that apply. -  Email']


QUESTION ITEM: [nan]


######## ADV_COMPLEX_MODES_UG ########
QUESTION STEM: ['Which of the following are acceptable modes of accessing 

In [324]:
qstem_str = DATA_SOURCE[DATA_SOURCE['Question Item Id'].str.contains(qstem, case=False)]['Question Stem'].unique()
qitem_str = DATA_SOURCE[DATA_SOURCE['Question Item Id'].str.contains(qstem, case=False)]['Question Item'].unique()[0]

# make sure there is only one question stem for each question item 
if (len(qstem_str) == 1) == False: 
    print ('!!!! ERROR: MULTIPLE QUESTION STEMS FOR ONE QUESTION STEM !!!!')
    # ex: the question item is not properly separated from stem 
    # ex: 'During this academic year (since the beginning of the Fall 21 semester), have you consulted with an academic advisor in your major or college? -  Help/reception desk, in-person'
    # instead of: 'During this academic year (since the beginning of the Fall 21 semester), have you consulted with an academic advisor in your major or college?' 

print(qstem_str), print(qitem_str)

['During this academic year (since the beginning of the Fall 21 semester), have you consulted with an academic advisor in your major or college?']
nan


(None, None)